# 2.1 Online Retail Data Set

In [1]:
from datetime import datetime
import pandas as pd
import mlxtend
import pandas
import timeit
import csv
import re
import sys

FILENAME = './datasets/online_retail.csv'

#### 2.1.1 Load the dataset

In [2]:
with open(FILENAME) as f:
    f.readline()
    
    items = []
    
    for item in csv.reader(f):
        is_item_valid = True
        
        if len(item) != 8:
            is_item_valid = False
        
        else:
            try:
                tmp_item = {}

                tmp_item['invoice_no']   = item[0]
                tmp_item['stock_code']   = item[1]
                tmp_item['description']  = item[2]
                tmp_item['quantity']     = int(item[3])
                tmp_item['invoice_date'] = datetime.strptime(item[4], "%m/%d/%Y %I:%M") # 2/6/2010 4:24 (Feb, 6)
                tmp_item['unit_price']   = float(item[5])
                tmp_item['customer_id']  = int(item[6])
                tmp_item['country']      = item[7]

                # If the invoice number starts with a "C" it means that the invoice has been canceled
                canceled_invoice_pattern = re.compile("(c|C).*")
                if canceled_invoice_pattern.match(tmp_item['invoice_no']):
                    is_item_valid = False
            
            except:
                is_item_valid = False

        if is_item_valid:
            items.append(tmp_item)

#### 2.1.2 Aggregating at "invoice" level

In [3]:
# Create a dictionary with the invoice number as key and a list of items as value
# While doing it, create a list of unique items
# item_id_dict: item code as key, int id as value
# unique_items: list of items, ordered by their id

next_item_id = 0
item_id_dict = {}
unique_items = []

invoices = {}
for item in items:
    invoice_no = item['invoice_no']
    item_code = item['stock_code']
    
    # Add invoice to the invoices list if not exists
    if invoice_no in invoices:
        invoice = invoices[invoice_no]
    else:
        invoice = {}
        invoice['items'] = []
        invoices[invoice_no] = invoice
    
    invoice['items'].append(item_code)
    
    # Unique items dictionary
    if not item_code in item_id_dict:
        item_id_dict[item_code] = next_item_id
        unique_items.append(item_code)
        next_item_id += 1

In [4]:
print(f"No of purchased items:\t{len(items)}")
print(f"No of items:\t\t{len(unique_items)}")
print(f"No of invoices:\t\t{len(invoices)}")

No of purchased items:	190176
No of items:		3453
No of invoices:		9611


#### 2.1.3 Matrix for DataFrame

In [5]:
# Create a matrix with N rows and M columns
# Every row is an invoice
# Every column is a product in the
# The cell is 1 if an invoice contains a certain element

# N = number of invoices
N = len(invoices)

# M = number of items
M = len(item_id_dict)

# Initialize PA matrix
pa_matrix = []

for invoice_no, invoice in invoices.items():
    """Following algorithm is too slow:"""
    # pa_row = [1 if item in invoice['items'] else 0 for item in unique_items]
    
    """Following algorithm is faster:"""
    # For each invoice, generate a row with all zeros
    pa_row = [0] * M
    
    # For each item purchased on the current invoice, set the corrisponding cell to "1" on the pa_row
    for item_in_invoice in invoice['items']:
        item_id = item_id_dict[item_in_invoice]
        pa_row[item_id] = 1
    
    # Append the PA row just generated to the PA matrix
    pa_matrix.append(pa_row)

# Convert pa_matrix to a DataFrame (df) with Pandas
df = pd.DataFrame(data=pa_matrix, columns = unique_items, dtype='int8')

In [6]:
df

,85123A,71053,84406B,84029G,84029E,22752,21730,22633,22632,84879,...,23448,90180B,84805B,90083,90089,46776B,85225,90210C,72783,23843
0,1,1,1,1,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1,1,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9606,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9607,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9608,0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,0
9609,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### 2.1.4 fpgrowth

In [7]:
from mlxtend.frequent_patterns import fpgrowth

# Minumum Support list
minsups = [0.5, 0.1, 0.05, 0.02, 0.01]

# Generate dictionary of Frequent Itemsets (FI) DataFrame
# Key: string with the minsup, Value: FI DataFrame
fi = {}

# fi[0.01] contains the FI with minsup = 0.01
for minsup in minsups:
    fi[minsup] = fpgrowth(df, minsup, use_colnames=True)

In [8]:
fi[0.01]

,support,itemsets
0,0.102903,(85123A)
1,0.016960,(84029G)
2,0.015815,(84029E)
3,0.015295,(22752)
4,0.013422,(71053)
...,...,...
916,0.010717,"(23344, 22086)"
917,0.011445,"(23293, 23296)"
918,0.010405,"(23294, 23293)"
919,0.011653,"(23295, 23293)"


In [9]:
# Print numbers of element for each minsup
for minsup in minsups:
    print(len(fi[minsup]), "items for minsup =", minsup)

0 items for minsup = 0.5
1 items for minsup = 0.1
18 items for minsup = 0.05
232 items for minsup = 0.02
921 items for minsup = 0.01


#### 2.1.5 minsup = 0.02

We have 232 items for minsup = 0.02.

We consider the most useful ones the sets with the higher number of items, e.g. "242  0.021040  (1945, 622, 617)"

#### 2.1.6 Extract relevant association rules

In [10]:
# Remove from the FI DataFrame rows:
# - which have just one item in the itemsets (we don't care about frequent items, but we're intrested about correlations)

# Put the result in fi_clean
fi_clean = {}

for minsup in minsups:
    items_to_drop = []

    for row_index, row in fi[minsup].iterrows():
        itemset = row['itemsets']

        if len(itemset) > 1:
            i = 1
        else:
            # Just one item in the itemsets
            items_to_drop.append(row_index)

    fi_clean[minsup] = fi[minsup].drop(items_to_drop)
    print(f"Dropped {len(items_to_drop)} items for minsup = {minsup} (the cleaned DF has {len(fi[minsup])} items)")

Dropped 0 items for minsup = 0.5 (the cleaned DF has 0 items)
Dropped 1 items for minsup = 0.1 (the cleaned DF has 1 items)
Dropped 18 items for minsup = 0.05 (the cleaned DF has 18 items)
Dropped 189 items for minsup = 0.02 (the cleaned DF has 232 items)
Dropped 561 items for minsup = 0.01 (the cleaned DF has 921 items)


In [11]:
# Sort the DF for DESC values of the support, so we have the best as first row
for minsup in minsups:
    fi_clean[minsup] = fi_clean[minsup].sort_values(by=['support'], ascending=False)

# Get the top 5 sets for each minsup
for minsup in minsups:
    if len(fi_clean[minsup]) > 1:
        print(f"Top 5 for minsup = {minsup}")
        display(fi_clean[minsup].head(5))
    
    else:
        print(f"No set found for minsup = {minsup}")

No set found for minsup = 0.5
No set found for minsup = 0.1
No set found for minsup = 0.05
Top 5 for minsup = 0.02


,support,itemsets
194,0.030798,"(22386, 85099B)"
220,0.029237,"(22697, 22699)"
189,0.027781,"(22727, 22726)"
210,0.027677,"(20725, 22383)"
203,0.027156,"(20725, 22384)"


Top 5 for minsup = 0.01


,support,itemsets
587,0.030798,"(22386, 85099B)"
769,0.029237,"(22697, 22699)"
568,0.027781,"(22727, 22726)"
727,0.027677,"(20725, 22383)"
686,0.027156,"(20725, 22384)"


In [12]:
def getItemFromCode(items, code):
    for item in items:
        if item['stock_code'] == code:
            return item
    return False

In [13]:
# Try to find the association rules from the frequent itemsets extracted with minsup = 0.01

print("Top 5 - frequently bought together")
# For each of the top 5 of FI (with minsup = 0.01)
for i in range(0, min(5, len(fi[0.01]))):
    print(f"Position no {i+1}:")
    most_frequent_set_list = list(fi_clean[0.01].iloc[i]["itemsets"])

    for item in most_frequent_set_list:
        freq_item = getItemFromCode(items, item)

        print("\t-", freq_item['description'])


Top 5 - frequently bought together
Position no 1:
	- JUMBO BAG PINK POLKADOT
	- JUMBO BAG RED RETROSPOT
Position no 2:
	- GREEN REGENCY TEACUP AND SAUCER
	- ROSES REGENCY TEACUP AND SAUCER 
Position no 3:
	- ALARM CLOCK BAKELIKE RED 
	- ALARM CLOCK BAKELIKE GREEN
Position no 4:
	- LUNCH BAG RED RETROSPOT
	- LUNCH BAG SUKI  DESIGN 
Position no 5:
	- LUNCH BAG RED RETROSPOT
	- LUNCH BAG PINK POLKADOT


#### 2.1.7 Association rules

We can do the same (and better, having antecedents and consequents) with the `mlxtend` library, by using `association_rules`

In [14]:
from mlxtend.frequent_patterns import association_rules

# Generate Association Rules (AR) DataFrame
ar = association_rules(fi[0.01], metric='confidence', min_threshold=0.8, support_only=False)

# Add the number of antecedents and consequents
ar["antecedents_len"] = ar["antecedents"].apply(lambda x: len(x))
ar["consequents_len"] = ar["consequents"].apply(lambda x: len(x))

display(ar)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,antecedents_len,consequents_len
0,"(22730, 22726)",(22727),0.014150,0.048174,0.011549,0.816176,16.942272,0.010868,5.177934,2,1
1,"(22697, 22423)",(22699),0.019977,0.043388,0.016752,0.838542,19.326676,0.015885,5.924824,2,1
2,(22746),(22748),0.013942,0.019041,0.012070,0.865672,45.464318,0.011804,7.302697,1,1
3,(22746),(22745),0.013942,0.018104,0.011861,0.850746,46.991508,0.011609,6.578701,1,1
4,"(22748, 22746)",(22745),0.012070,0.018104,0.010717,0.887931,49.045432,0.010498,8.761531,2,1
5,"(22746, 22745)",(22748),0.011861,0.019041,0.010717,0.903509,47.451491,0.010491,10.166306,2,1
6,(22698),(22697),0.029654,0.036937,0.024243,0.817544,22.133561,0.023148,5.278327,1,1
7,(22698),(22699),0.029654,0.043388,0.023827,0.803509,18.519239,0.022540,4.868473,1,1
8,"(22698, 22423)",(22697),0.016231,0.036937,0.014255,0.878205,23.775858,0.013655,7.907255,2,1
9,"(22698, 22423)",(22699),0.016231,0.043388,0.013838,0.852564,19.649865,0.013134,6.488326,2,1


In [15]:
# Take the top 10 and iterate each element
ar = ar.head(10)

for row in ar.iterrows():
    
    antecedents = row[1]['antecedents']
    consequents = row[1]['consequents']
    
    print("Antecedents:")
    for item_code in antecedents:
        print(f"\t{getItemFromCode(items, item_code)['description']}")
              
    print("Consequents:")
    for item_code in consequents:
        print(f"\t{getItemFromCode(items, item_code)['description']}")

    print("\n")

Antecedents:
	ALARM CLOCK BAKELIKE IVORY
	ALARM CLOCK BAKELIKE GREEN
Consequents:
	ALARM CLOCK BAKELIKE RED 


Antecedents:
	GREEN REGENCY TEACUP AND SAUCER
	REGENCY CAKESTAND 3 TIER
Consequents:
	ROSES REGENCY TEACUP AND SAUCER 


Antecedents:
	POPPY'S PLAYHOUSE LIVINGROOM 
Consequents:
	POPPY'S PLAYHOUSE KITCHEN


Antecedents:
	POPPY'S PLAYHOUSE LIVINGROOM 
Consequents:
	POPPY'S PLAYHOUSE BEDROOM 


Antecedents:
	POPPY'S PLAYHOUSE KITCHEN
	POPPY'S PLAYHOUSE LIVINGROOM 
Consequents:
	POPPY'S PLAYHOUSE BEDROOM 


Antecedents:
	POPPY'S PLAYHOUSE LIVINGROOM 
	POPPY'S PLAYHOUSE BEDROOM 
Consequents:
	POPPY'S PLAYHOUSE KITCHEN


Antecedents:
	PINK REGENCY TEACUP AND SAUCER
Consequents:
	GREEN REGENCY TEACUP AND SAUCER


Antecedents:
	PINK REGENCY TEACUP AND SAUCER
Consequents:
	ROSES REGENCY TEACUP AND SAUCER 


Antecedents:
	PINK REGENCY TEACUP AND SAUCER
	REGENCY CAKESTAND 3 TIER
Consequents:
	GREEN REGENCY TEACUP AND SAUCER


Antecedents:
	PINK REGENCY TEACUP AND SAUCER
	REGENCY CAKESTA

#### 2.1.8 Apriori Algorithm

In [16]:
from mlxtend.frequent_patterns import apriori

# Generate dictionary of Frequent Itemsets (FI) DataFrame
# Key: string with the minsup, Value: FI DataFrame
fi_apriori = {}

for minsup in minsups:
    try:
        fi_apriori[minsup] = apriori(df, minsup)
    except:
        print(f"Error while calculating the FI with Apriori algorithm of minsup = {minsup}")

Error while calculating the FI with Apriori algorithm of minsup = 0.01


In [17]:
ar_fpgrowth = association_rules(fi[0.02], metric='confidence', min_threshold=0.8, support_only=False)
ar_apriori = association_rules(fi_apriori[0.02], metric='confidence', min_threshold=0.8, support_only=False)

print("FP-Growth")
display(ar_fpgrowth)

print("Apriori")
display(ar_apriori)

FP-Growth


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(22698),(22697),0.029654,0.036937,0.024243,0.817544,22.133561,0.023148,5.278327
1,(22698),(22699),0.029654,0.043388,0.023827,0.803509,18.519239,0.022540,4.868473
2,"(22698, 22697)",(22699),0.024243,0.043388,0.021330,0.879828,20.278250,0.020278,7.960380
3,"(22698, 22699)",(22697),0.023827,0.036937,0.021330,0.895197,24.235869,0.020450,9.189228


Apriori


,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1551),(617),0.029654,0.043388,0.023827,0.803509,18.519239,0.022540,4.868473
1,(1551),(622),0.029654,0.036937,0.024243,0.817544,22.133561,0.023148,5.278327
2,"(617, 1551)",(622),0.023827,0.036937,0.021330,0.895197,24.235869,0.020450,9.189228
3,"(622, 1551)",(617),0.024243,0.043388,0.021330,0.879828,20.278250,0.020278,7.960380


Using Apriori or FP-Growth, in this case, the result doesn't change.

In [37]:
print("FP-Growth")
display(timeit.timeit(lambda: fpgrowth(df, 0.1), number=1))

print("Apriori")
display(timeit.timeit(lambda: apriori(df, 0.1), number=1))


FP-Growth


0.6120047

Apriori


0.2705355999999881